In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- RR summary ---
    if "RR" in d.columns:
        or_mean = d["RR"].mean()
        or_std = d["RR"].std(ddof=1)
        df_out.loc[feature, "RR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minRR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxRR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["RR"].mean()
    or_std  = d["RR"].std(ddof=1)
    df_out.loc[idx, "RR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minRR"] = d["2.5%"].min()
    df_out.loc[idx, "maxRR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
import time

t0 = time.perf_counter()






In [4]:
data = pd.read_parquet('data/BBAG-long.parquet')

In [5]:
data = data[data.country != 'Slovakia'].reset_index(drop =  True)

In [6]:
Counter(data.country)

Counter({'Austria': 4320,
         'Belgium': 4942,
         'Czechia': 5575,
         'Switzerland': 2634,
         'Germany': 4567,
         'Denmark': 2609,
         'Estonia': 7522,
         'Spain': 4912,
         'France': 3640,
         'Hungary': 3010,
         'Italy': 4541,
         'Netherlands': 2546,
         'Poland': 3612,
         'Portugal': 1914,
         'Sweden': 2792,
         'Slovenia': 5291,
         'Luxembourg': 2104,
         'Greece': 2599,
         'Croatia': 2798,
         'Bulgaria': 1953,
         'Cyprus': 1205,
         'Finland': 1982,
         'Lithuania': 2033,
         'Latvia': 1684,
         'Malta': 1259,
         'Romania': 2083})

In [7]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


# Relative Risk

## Without co-vars

### Without iterations

In [8]:
results_merge_df_all = data.copy()

results_merge_df_all = results_merge_df_all.loc[:, ~results_merge_df_all.columns.duplicated()]

df_directions_odd = pd.DataFrame()

for i in vars_:
    
    y_ols = results_merge_df_all['GAP_bin']

    
    X_ols = results_merge_df_all[[i] + ['delta_time']]
    X_train, X_test, y_train, y_test = train_test_split(X_ols, y_ols, test_size=0.2, random_state=42)
    

    scaler = MinMaxScaler((0.05, 0.95))
    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    X_train_scaled['intercept'] = 1
    X_test_scaled['intercept'] = 1
    
    model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)
    
    params = model.params
    conf = np.exp(model.conf_int())
    conf['RR'] = np.exp(params)
    conf['z'] =model.tvalues
    conf['P>|z|'] =model.pvalues
    conf.columns = ['2.5%', '97.5%', 'RR', 'z', 'P>|z|']
    
    df = conf.loc[i:i]
    df['Feature'] = i
    df_directions_odd = pd.concat([df_directions_odd, df])

df_directions_odd = df_directions_odd.reset_index(drop=True)

df_directions_odd.to_excel('Results/long/long_RR_-removing-slovakia.xlsx')
df_directions_odd

,2.5%,97.5%,RR,z,P>|z|,Feature
0,1.363941,1.460328,1.411312,19.777799,4.624403e-87,Mono
1,0.867263,0.948238,0.906847,-4.294017,1.754689e-05,One
2,0.769138,0.827344,0.797711,-12.144517,6.134382e-34,Two
3,0.678439,0.750525,0.713572,-13.100502,3.270891e-39,Three
4,0.684083,0.732499,0.707877,-19.804605,2.716841e-87,Total


In [12]:
np.exp(params)

Total         0.707877
delta_time    0.718641
intercept     0.675632
dtype: float64

### 1000-iteration

In [9]:


count = 0

results_dict = {}
for i in vars_:
    results_dict[i + "_y"] = pd.Series(dtype=float)
    results_dict[i + "_ypred"] = pd.Series(dtype=float)

results_merge_df_all = data.copy()


df_directions_odd = pd.DataFrame()

n_boosts = 10  
test_size = 500 / results_merge_df_all.shape[0]

for boosts in range(n_boosts):
    for i in vars_:

        y_ols = results_merge_df_all["GAP_bin"]
        X_ols = results_merge_df_all[[i] + ['delta_time']]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols,
            test_size=test_size,
            stratify=results_merge_df_all["GAP_bin"],
            random_state=boosts
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled["intercept"] = 1
        X_test_scaled["intercept"] = 1


        model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)


        y_test_pred = model.predict(X_test_scaled)

        results_dict[i + "_y"] = pd.concat([results_dict[i + "_y"], y_test], axis=0)
        results_dict[i + "_ypred"] = pd.concat([results_dict[i + "_ypred"], y_test_pred], axis=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf["RR"] = np.exp(params)
        conf["z"] = model.tvalues
        conf["P>|z|"] = model.pvalues
        conf.columns = ["2.5%", "97.5%", "RR", "z", "P>|z|"]

        df = conf.loc[i:i].copy()
        df["Feature"] = i

        hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
        hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
        hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

        df["hl_statistic_10"] = hl10_stat
        df["hl_pvalue_10"] = hl10_p

        df["hl_statistic_50"] = hl50_stat
        df["hl_pvalue_50"] = hl50_p

        df["hl_statistic_100"] = hl100_stat
        df["hl_pvalue_100"] = hl100_p

        df_directions_odd = pd.concat([df_directions_odd, df], axis=0)

    df_directions_odd = df_directions_odd.reset_index(drop=True)

df_directions_odd;


In [10]:
df_directions_odd_f = pd.DataFrame()

for f in vars_:
    df_directions_odd_f = summarize_feature(df_directions_odd, f, df_directions_odd_f, stat_cols=(10, 50, 100))

df_directions_odd_f.to_excel('Results/long/long_RR_-removing-slovakia-1000-iter.xlsx')
df_directions_odd_f

,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR
Mono,0.273457,0.476525,0.996208,1.423729,1.422248,1.425210,1.378999,1.470514
One,0.249304,0.505847,0.994659,0.908238,0.906653,0.909824,0.869937,0.947949
Two,0.134957,0.416494,0.994437,0.794455,0.793505,0.795405,0.767513,0.822541
Three,0.149278,0.303391,0.987923,0.703193,0.701963,0.704422,0.669643,0.737397
Total,0.274497,0.469258,0.996701,0.701711,0.700980,0.702442,0.679351,0.724507


## Odd ratios with macrosocials

### Without Iteration

In [15]:
vars_social_physical = ['gender_equal_l', 'Polution_conc_inv', 'Eq']

vars_sociopolitical = ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est']

In [18]:
[[i, covar] + ['delta_time', 'GAP_bin']]

[['Mono', 'total_exposomes', 'delta_time', 'GAP_bin']]

In [19]:
results_merge_df_all = data.copy()

covar_list = (
    ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP',
     'number_leng_inst', 'number_stable_inst', 'distance', 'Migration']
    + vars_social_physical
    + vars_sociopolitical
)

df_directions_odd_all = []
df_directions_cov_odd_all = []

for covar in covar_list:
    for i in vars_:

        c_results_merge_df_all = results_merge_df_all.dropna(subset= [i, covar] + ['delta_time', 'GAP_bin'])
        c_results_merge_df_all.reset_index(drop = True, inplace = True)
        
        y_ols = c_results_merge_df_all['GAP_bin']
        X_ols = c_results_merge_df_all[[i, covar] + ['delta_time']]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['RR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'RR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar
        df_directions_odd_all.append(df_i)

        df_c = conf.loc[covar:covar].copy()
        df_c['Feature'] = f"{i}({covar})"
        df_c['Exposure'] = i          # opcional pero útil
        df_c['Covar'] = covar
        df_directions_cov_odd_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [20]:
for covar in ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP', 'number_leng_inst', 'number_stable_inst', 'distance', 'Migration'] + vars_social_physical + vars_sociopolitical:
    c_dfRR = df_directions_odd[df_directions_odd.Covar == covar]
    c_dfRR.to_excel('Results/long/long_RR_-removing-slovakia_covar-' + covar + '.xlsx')

    print(covar)
    display(c_dfRR)

total_exposomes


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
0,1.348949,1.447096,1.397161,18.666273,9.312885e-78,Mono,total_exposomes
1,0.872320,0.953587,0.912049,-4.051378,5.091681e-05,One,total_exposomes
2,0.782526,0.842907,0.812156,-10.972633,5.174342e-28,Two,total_exposomes
3,0.693682,0.768113,0.729949,-12.106267,9.784664e-34,Three,total_exposomes
4,0.690342,0.740642,0.715050,-18.694237,5.515354e-78,Total,total_exposomes


sociopolitical


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
5,1.363623,1.460090,1.411032,19.746314,8.629041e-87,Mono,sociopolitical
6,0.864800,0.945786,0.904387,-4.400732,1.078863e-05,One,sociopolitical
7,0.768819,0.827314,0.797531,-12.093840,1.138392e-33,Two,sociopolitical
8,0.678510,0.750633,0.713661,-13.090474,3.732719e-39,Three,sociopolitical
9,0.684198,0.732671,0.708020,-19.773295,5.056317e-87,Total,sociopolitical


social_physical


...

,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
60,1.362431,1.459049,1.409913,19.654197,5.322499e-86,Mono,inclu_suff_est
61,0.865524,0.946568,0.905139,-4.364816,1.272298e-05,One,inclu_suff_est
62,0.771694,0.830477,0.800546,-11.878588,1.529279e-32,Two,inclu_suff_est
63,0.679213,0.751596,0.714488,-13.013944,1.019419e-38,Three,inclu_suff_est
64,0.684686,0.733315,0.708584,-19.680575,3.163936e-86,Total,inclu_suff_est


cred_elect_est


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
65,1.354006,1.448928,1.400663,19.493489,1.246826e-84,Mono,cred_elect_est
66,0.875485,0.957571,0.915608,-3.856254,1.151377e-04,One,cred_elect_est
67,0.765910,0.823504,0.794185,-12.458672,1.254420e-35,Two,cred_elect_est
68,0.679285,0.751149,0.714314,-13.114016,2.737129e-39,Three,cred_elect_est
69,0.689484,0.737893,0.713278,-19.519010,7.569093e-85,Total,cred_elect_est


local_dem_est


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
70,1.346728,1.442447,1.393766,18.954244,4.073645e-80,Mono,local_dem_est
71,0.878981,0.961578,0.919352,-3.669993,2.425567e-04,One,local_dem_est
72,0.774873,0.833481,0.803643,-11.752504,6.855660e-32,Two,local_dem_est
73,0.690177,0.762970,0.725662,-12.536125,4.735944e-36,Three,local_dem_est
74,0.692587,0.741888,0.716814,-18.979371,2.526024e-80,Total,local_dem_est


### 1000-Iteration

In [21]:

results_merge_df_all = data.copy()

covar_list = (
    ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP',
     'number_leng_inst', 'number_stable_inst', 'distance', 'Migration']
    + vars_social_physical
    + vars_sociopolitical
)



n_boosts = 10  
test_size = 500 / results_merge_df_all.shape[0]


df_directions_odd_all = []
df_directions_cov_all = []


results_dict = {}  

for covar in covar_list:
    print(covar)
    for boosts in range(n_boosts):

        for i in vars_:

            c_results_merge_df_all = results_merge_df_all.dropna(subset= [i, covar] + ['delta_time', 'GAP_bin'])
            c_results_merge_df_all.reset_index(drop = True, inplace = True)
            
            y_ols = c_results_merge_df_all['GAP_bin']
            X_ols = c_results_merge_df_all[[i, covar] + ['delta_time']]

            # Split igual al primero: test fijo ~500 y estratificado
            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar, i, "y")
            key_p = (covar, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["RR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "RR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)

            df_c = conf.loc[covar:covar].copy()
            df_c["Feature"] = i + "(" + covar + ")"
            df_c["Exposure"] = i
            df_c["Covar"] = covar
            df_c["boosts"] = boosts

            df_c["hl_statistic_10"] = hl10_stat
            df_c["hl_pvalue_10"] = hl10_p
            df_c["hl_statistic_50"] = hl50_stat
            df_c["hl_pvalue_50"] = hl50_p
            df_c["hl_statistic_100"] = hl100_stat
            df_c["hl_pvalue_100"] = hl100_p

            df_directions_cov_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


total_exposomes
sociopolitical
social_physical
GDP
number_leng_inst
number_stable_inst
distance
Migration
gender_equal_l
Polution_conc_inv
Eq
free_parties_l
inclu_suff_est
cred_elect_est
local_dem_est


In [22]:

features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100),
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [23]:
for covar in ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP', 'number_leng_inst', 'number_stable_inst', 'distance', 'Migration'] + vars_social_physical + vars_sociopolitical:
    c_dfRR = df_directions_odd_f[df_directions_odd_f.Covar == covar]
    c_dfRR.to_excel('Results/long/long_RR_-removing-slovakia_covar-' + covar + '-1000-iter.xlsx')

    print(covar)
    display(c_dfRR)

total_exposomes


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
0,total_exposomes,Mono,0.197652,0.405728,0.937986,1.410730,1.408957,1.412503,1.364508,1.458717,10.0
1,total_exposomes,One,0.169668,0.346444,0.949427,0.913510,0.911880,0.915141,0.875000,0.953386,10.0
2,total_exposomes,Two,0.128380,0.211598,0.875195,0.808506,0.807430,0.809581,0.780567,0.838042,10.0
3,total_exposomes,Three,0.124685,0.171162,0.887723,0.718851,0.717558,0.720144,0.684256,0.753783,10.0
4,total_exposomes,Total,0.201840,0.481212,0.952311,0.708181,0.707290,0.709073,0.684850,0.732205,10.0


sociopolitical


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
5,sociopolitical,Mono,0.286313,0.475396,0.996756,1.423078,1.421635,1.424520,1.378472,1.469903,10.0
6,sociopolitical,One,0.226214,0.536795,0.995769,0.905064,0.903450,0.906678,0.866805,0.944650,10.0
7,sociopolitical,Two,0.137940,0.400411,0.995033,0.794742,0.793825,0.795659,0.767728,0.822807,10.0
8,sociopolitical,Three,0.250247,0.404975,0.991897,0.703339,0.702110,0.704568,0.669772,0.737606,10.0
9,sociopolitical,Total,0.286365,0.469227,0.996527,0.702033,0.701320,0.702746,0.679635,0.724785,10.0


social_physical


...

,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
60,inclu_suff_est,Mono,0.253699,0.513460,0.996658,1.422591,1.421095,1.424088,1.377825,1.469582,10.0
61,inclu_suff_est,One,0.195203,0.408538,0.995013,0.906512,0.904920,0.908103,0.868227,0.946238,10.0
62,inclu_suff_est,Two,0.227200,0.451549,0.997358,0.796930,0.795989,0.797871,0.769765,0.825135,10.0
63,inclu_suff_est,Three,0.165906,0.445370,0.994196,0.704006,0.702760,0.705253,0.670327,0.738450,10.0
64,inclu_suff_est,Total,0.253794,0.510548,0.996863,0.702275,0.701535,0.703015,0.679784,0.725127,10.0


cred_elect_est


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
65,cred_elect_est,Mono,0.224145,0.577392,0.997505,1.414104,1.412590,1.415618,1.369953,1.460362,10.0
66,cred_elect_est,One,0.235866,0.356132,0.992515,0.916052,0.914382,0.917722,0.877161,0.956269,10.0
67,cred_elect_est,Two,0.164224,0.369671,0.991347,0.791259,0.790329,0.792188,0.764606,0.819056,10.0
68,cred_elect_est,Three,0.042340,0.303074,0.986976,0.703829,0.702620,0.705038,0.670388,0.737875,10.0
69,cred_elect_est,Total,0.224084,0.565020,0.997114,0.706503,0.705745,0.707261,0.684088,0.729307,10.0


local_dem_est


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
70,local_dem_est,Mono,0.192217,0.438225,0.995949,1.407311,1.405720,1.408902,1.362621,1.453853,10.0
71,local_dem_est,One,0.129891,0.396671,0.992209,0.920576,0.918921,0.922231,0.881466,0.961087,10.0
72,local_dem_est,Two,0.147024,0.323044,0.990952,0.800009,0.799022,0.800996,0.772852,0.828353,10.0
73,local_dem_est,Three,0.114907,0.449594,0.989701,0.714403,0.713127,0.715679,0.680475,0.748836,10.0
74,local_dem_est,Total,0.189004,0.439056,0.995631,0.709920,0.709116,0.710725,0.687158,0.733240,10.0


In [ ]:
dt = time.perf_counter() - t0
m, s = divmod(dt, 60)
print(f"Tiempo total: {int(m)} min {s:.1f} s")